In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Feb 27 12:00:25 2019

@author: robertgrant
"""

import numpy as np
from databaker.framework import *
import pandas as pd
from databakerUtils.writers import v4Writer
import re
import requests
import json
import datetime
import math
from datetime import datetime
import glob
import os


file = 'IPHRP INDEX CHART DATA.csv'

df = pd.read_csv(file)

#isolate 12m growth
pc = df.copy(deep = True)
val = df.copy(deep = True)

#create value col 
pc['value'] = pc['12m growth']
val['value'] = val['Index value']

#explainer col
pc['iphrp-variable'] = 'year-on-year-change'
val['iphrp-variable'] = 'index'

pc['variable'] = 'Year-on-year change'
val['variable'] = 'Index'


#concatenatae
full = pd.concat([pc, val])

#tidy
full['mmm-yy'] = full['Date']

#change NI code
full['RegionCode'] = full['RegionCode'].replace('N92000001','N92000002')

#add data marking
full['Data Marking'] = ''

full.loc[full['value'] == '-', 'Data Marking'] = '.'
full.loc[full['value'] == '-', 'value'] = ''

v4 = full[['value','Data Marking','mmm-yy','Date','RegionCode','Geography','iphrp-variable','variable']]
v4.columns = ['V4_1','Data Marking','mmm-yy','Time','administrative-geography','Geography','index-and-year-change','IndexAndYearChange']

def TimeCorrector(value):
    if '-' in value:
        return value
    else:
        return value[:3] + '-' + value[5:]

v4['mmm-yy'] = v4['mmm-yy'].apply(TimeCorrector)
v4['Time'] = v4['mmm-yy']

output_file = 'v4-iphrp-' + v4['mmm-yy'][-1:].item().lower() + '.csv'
v4.to_csv(output_file, index = False) 

from databakerUtils.sparsityFunctions import SparsityFiller

SparsityFiller(output_file)